In [8]:
import numpy as np
from collections import Counter
from nltk import ngrams

BLEU(Bilingual Evaluation Understudy)  
- 기계 번역 결과와 사람이 번역한 결과가 얼마나 유사한지 비교하여 성능을 측정하는 방법
- 측정 기준은 n-gram에 기반함
- 언어에 구애받지 않고 사용할 수 있으며 계산 속도가 빠름

단어 개수 카운트로 측정하기(Unigram Precision)  
- 두 개의 기계 번역기에 같은 한국어 문장을 입력하여 번역된 영어 문장의 성능을 측정하고자 할 때  
  - Candidate 1, 2 : 번역된 문장. 이 문장의 성능을 평가하기 위해서는 정답으로 비교되는 문장이 있어야 함
  - Reference 1, 2, 3 : 세 명의 사람에게 한국어를 보고 영작하여 만든 세 개의 번역 문장
- Example 1
  - Candidate 1 : It is a guide to action which ensures that the military always obeys the commands of the party.
  - Reference 1 : It is a guide to action that ensures that the military will forever heed Party commands.
  - Reference 2 : It is the guiding principle which guarantees the military forces always being under the command of the Party.
  - Reference 3 : It is the practical guide for the army always to heed the directions of the party.
    - Candidate를 Ca로, Reference를 Ref로 축약
    - Ca 1, 2를 Ref 1, 2, 3과 비교하여 성능을 측정하고자 함. 가장 직관적인 성능 평가 방법은 Ref 1, 2, 3 중 어느 한 문장이라도 등장한 단어의 개수를 Ca에서 세는 것. 그리고 그 후에 Ca의 모든 단어의 카운트의 합. 즉, Ca에서의 총 단어의 수으로 나눠줍니다.
- 이러한 측정 방법을 유니그램 정밀도(Unigram Precision)라고 함  
$$\text{Unigram Precision =}\frac{\text{Ref들 중에서 존재하는 Ca의 단어의 수}}{\text{Ca의 총 단어 수}} = \frac{\text{the number of Ca words(unigrams) which occur in any Ref}}{\text{the total number of words in the Ca}}$$
- 각각의 Ca1과 Ca2의 유니그램 정밀도 값
  - Ca 1 Unigram Precision = 17/18
  - Ca 2 Unigram Precision = 8/14

중복을 제거하여 보정하기(Modified Unigram Precision)  
- Candidate : the the the the the the the
- Reference 1 : the cat is on the mat
- Reference 2 : there is a cat on the mat
  - 이 번역은 앞서 유니그램 정밀도에 따르면 7/7=1 이라는 최고의 성능 평가를 받게 됨
  - 이에 유니그램 정밀도를 다소 보정할 필요가 있음
  - 정밀도의 분자를 계산하기 위해 Ref와 매칭하며 카운트하는 과정에서 Ca의 유니그램이 이미 Ref에서 매칭된 적이 있었는지를 고려해야 함 
- 우선, 유니그램이 하나의 Ref에서 최대 몇 번 등장했는지를 카운트함
  - Max_Ref_Count : maximum reference count 
  - Max_Ref_Count 가 기존의 단순 카운트한 값보다 작은 경우에는 이 값을 최종 카운트 값으로 대체함
  - Count_clip = min(Count, Max_Ref_Count)
  - 이를 보정된 유니그램 정밀도(Modified Unigram Precision)라고 함

$\text{Modified Unigram Precision =}\frac{\text{Ca의 각 유니그램에 대해 }Count_{clip}\text{을 수행한 값의 총 합}}{\text{Ca의 총 유니그램 수}}=\frac{\sum_{unigram∈Candidate}\ Count_{clip}(unigram)}
{\sum_{unigram∈Candidate}\ Count(unigram)}$

In [4]:
# 토큰화 된 문장에서 n-gram 카운트
def simple_count(tokens, n):
    return Counter(ngrams(tokens, n))

candidate = "It is a guide to action which ensures that the military always "\
            "obeys the commands of the party."
tokens = candidate.split()
result = simple_count(tokens, 1) # n=1은 unigram
print('example 1 result : ', result)

candidate = 'the the the the the the the'
tokens = candidate.split() # 토큰화
result = simple_count(tokens, 1) # n = 1은 유니그램
print('example 2 result : :', result)

example 1 result :  Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})
example 2 result : : Counter({('the',): 7})


In [6]:
def count_clip(candidate, reference_list, n):
  # Ca 문장에서 n-gram 카운트
  ca_cnt = simple_count(candidate, n)
  max_ref_cnt_dict = dict()

  for ref in reference_list: 
    # Ref 문장에서 n-gram 카운트
    ref_cnt = simple_count(ref, n)

    # 각 Ref 문장에 대해서 비교하여 n-gram의 최대 등장 횟수를 계산.
    for n_gram in ref_cnt: 
      if n_gram in max_ref_cnt_dict:
        max_ref_cnt_dict[n_gram] = max(ref_cnt[n_gram], max_ref_cnt_dict[n_gram])
      else:
        max_ref_cnt_dict[n_gram] = ref_cnt[n_gram]

  return {
      n_gram: min(ca_cnt.get(n_gram, 0), max_ref_cnt_dict.get(n_gram, 0)) \
            for n_gram in ca_cnt}

candidate = 'the the the the the the the'
references = [
    'the cat is on the mat',
    'there is a cat on the mat']
result = count_clip(
    candidate.split(), list(map(lambda ref: ref.split(), references)), 1)
print('보정된 유니그램 카운트 :', result)

def modified_precision(candidate, reference_list, n):
    clip_cnt = count_clip(candidate, reference_list, n)
    total_clip_cnt = sum(clip_cnt.values()) # 분자

    cnt = simple_count(candidate, n)
    total_cnt = sum(cnt.values()) # 분모
    
    if total_cnt == 0: # 분모가 0이 되는 것을 방지
        total_cnt = 1

    # 분자 : count_clip의 합, 분모 : 단순 count의 합 ==> 보정된 정밀도
    return (total_clip_cnt / total_cnt)

result = modified_precision(
    candidate.split(), list(map(lambda ref: ref.split(), references)), n=1)
print('보정된 유니그램 정밀도 :',result)

보정된 유니그램 카운트 : {('the',): 2}
보정된 유니그램 정밀도 : 0.2857142857142857


순서를 고려하기 위해서 n-gram으로 확장하기  
- BoW 표현과 유사하게 유니그램 정밀도와 같이 각 단어의 빈도수로 접근하는 방법은 단어의 순서를 고려하지 않는다는 특징이 있음
- Example 1
  - Candidate 1 : It is a guide to action which ensures that the military always obeys the commands of the party.
  - Candidate 2 : It is to insure the troops forever hearing the activity guidebook that party direct.
  - Candidate 3 : the that military a is It guide ensures which to commands the of action obeys always party the.
  - Reference 1 : It is a guide to action that ensures that the military will forever heed Party commands.
  - Reference 2 : It is the guiding principle which guarantees the military forces always being under the command of the Party.
  - Reference 3 : It is the practical guide for the army always to heed the directions of the party.
- Ca 3은 Ca 1에서 모든 유니그램의 순서를 랜덤으로 섞은 실제 영어 문법에 맞지 않은 문장
- Ref 1, 2, 3과 비교하여 유니그램 정밀도를 적용하면 Ca1과 Ca3의 두 정밀도는 동일함
- 유니그램 정밀도는 유니그램의 순서를 전혀 고려하지 않기 때문
- 이에 대안으로 개별적인 단어로서 카운트하는 유니그램 정밀도에서 다음에 등장한 단어까지 함께 고려하여 카운트하도록 유니그램 외에도 Bigram, Trigram, 4-gram 단위 등으로 계산한 정밀도. n-gram을 이용한 정밀도를 도입함  
$BLEU = exp(\sum_{n=1}^{N}w_{n}\ \text{log}\ p_{n})$  
- P_n : 각 gram의 보정된 정밀도
- N : n-gram에서 n의 최대 숫자. 보통은 4. N이 4라는 것은 p1, p2, p3, p4.
- W_n : 각 gram의 보정된 정밀도에 서로 다른 가중치를 줄 수 있음. 이 가중치의 합은 1로 함. 예를 들어 
N이 4라고 하였을 때, p1, p2, p3, p4에 대해서 동일한 가중치를 주고자한다면 모두 0.25를 적용할 수 있음

짧은 문장 길이에 대한 패널티(Brevity Penalty)  
- n-gram으로 단어의 순서를 고려한다고 하더라도 바로 Ca의 길이에 BLEU의 점수가 과한 영향을 받을 수 있음
- Example 1
  - Candidate4 : it is
  - 이 문장은 유니그램 정밀도나 바이그램 정밀도가 각각 2/2, 1/1로 두 정밀도 모두 1이라는 높은 정밀도를 얻음
  - Ca가 Ref보다 문장의 길이가 짧은 경우에는 점수에 패널티를 줄 필요가 있음
  - 이를 브레버티 패널티(Brevity Penalty)라고 함.(짧음 패널티) 

- 만약 반대로 Ca의 길이가 Ref보다 긴 경우
- Example 3
  - Candidate 1: I always invariably perpetually do.
  - Candidate 2: I always do.
  - Reference 1: I always do.
  - Reference 2: I invariably do.
  - Reference 3: I perpetually do.
- Example 3에서 Ca 1은 가장 많은 단어를 사용했지만 Ca 2보다 좋지 못한 번역이므로 Ref의 단어를 가장 많이 사용한 것이 꼭 좋은 번역이라는 의미는 아님
- Ca의 길이가 불필요하게 Ref보다 긴 경우에는 BLEU 수식에서 정밀도를 n-gram으로 확장하여 바이그램, 트라이그램 정밀도 등을 모두 계산에 사용하고 있는 것만으로도 이미 패널티를 받고 있음
- 브레버티 패널티는 BLEU의 식에 곱하는 방식으로 사용  

$$ BLEU = BP × exp(\sum_{n=1}^{N}w_{n}\ \text{log}\ p_{n}) $$   
- 위의 수식은 패널티를 줄 필요가 없는 경우에는 BP의 값이 1이어야 함  
$$ BP = \begin{cases}1&\text{if}\space c>r\\ e^{(1-r/c)}&\text{if}\space c \leq r \end{cases} $$  
c : Candidate의 길이  
r : Candidate와 가장 길이 차이가 작은 Reference의 길이  

In [10]:
# Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
def closest_ref_length(candidate, reference_list):
    ca_len = len(candidate) # ca 길이
    ref_lens = (len(ref) for ref in reference_list) # Ref들의 길이
    # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
    closest_ref_len = min(
        ref_lens, key=lambda ref_len: (abs(ref_len - ca_len), ref_len))
    return closest_ref_len

def brevity_penalty(candidate, reference_list):
    ca_len = len(candidate)
    ref_len = closest_ref_length(candidate, reference_list)

    if ca_len > ref_len:
        return 1
    elif ca_len == 0 : # candidate가 비어있다면 BP = 0 -> BLEU = 0.0
        return 0
    else:
        return np.exp(1 - ref_len/ca_len)

def bleu_score(candidate, reference_list, weights=[0.25, 0.25, 0.25, 0.25]):
    bp = brevity_penalty(candidate, reference_list) # 브레버티 패널티, BP

    p_n = [modified_precision(candidate, reference_list, n=n) \
        for n, _ in enumerate(weights,start=1)] 
    # p1, p2, p3, ..., pn
    score = np.sum(
        [w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
    return bp * np.exp(score)

In [12]:
import nltk.translate.bleu_score as bleu

candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'
]

print('psudo BLEU :', bleu_score(
    candidate.split(),list(map(lambda ref: ref.split(), references))))
print('nltk BLEU :', bleu.sentence_bleu(
    list(map(lambda ref: ref.split(), references)),candidate.split()))

psudo BLEU : 0.5045666840058485
nltk BLEU : 0.5045666840058485
